In [1]:
import sklearn
import sys
print(f'sklearn version : {sklearn.__version__}')
print(f'python version : {sys.version}')

sklearn version : 0.23.2
python version : 3.7.8 | packaged by conda-forge | (default, Jul 31 2020, 01:53:57) [MSC v.1916 64 bit (AMD64)]


In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=RuntimeWarning)
from IPython.display import HTML, display # jupyter 사진넣기
import time                      # excution time 계산
from datetime import datetime   # system time 
import json                     # json save
# from imblearn.over_sampling import SMOTE
from sklearn.tree import export_graphviz  # 결정트리 모형
# import graphviz                         # 결정트리 모형
from sklearn import tree                  # 결정트리 모형  # Classifier tree
from sklearn.inspection import permutation_importance  # feature 중요도
from sklearn.pipeline import Pipeline
from scipy import stats
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Binarizer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             confusion_matrix, precision_recall_curve, roc_curve)
from sklearn.model_selection import learning_curve, ShuffleSplit

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import (VotingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                                ExtraTreesClassifier, GradientBoostingClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# import xgboost as xgb                # python wrapper
from xgboost import XGBClassifier    # sklearn wrapper
from lightgbm import LGBMClassifier  # sklearn wrapper

import pickle                         # 모델 저장
import matplotlib as mpl              # 한글깨짐
import matplotlib.font_manager as fm  # 한글깨짐
import matplotlib.font_manager        # 한글깨짐

### Plot

In [3]:
# linux 
font_dirs = ['/home/sch/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
font_list = fm.createFontList(font_files)
fm.fontManager.ttflist.extend(font_list)
[f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Nanum' in f.name][:3]

[]

In [4]:
# 모델에 대한 성능평가지표개선을 위한 threshold 선택관련 plot
def precision_recall_curve_plot(y_test, pos_proba, data_name=None, model_name=None,time_name=None ): # pred_proba positive값
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출.
    precisions, recalls, thresholds = precision_recall_curve(y_test, pos_proba)

    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    fig = plt.figure(figsize=(8, 6))
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
    
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary],linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')

    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))

    # x축, y축 label과 legend, 그리고 grid 설정
    plt.title(f"Precision and Recall Trade off Curve {data_name} {model_name}")
    plt.xlabel('Threshold value)')
    plt.ylabel('Precision and Recall value')
    plt.legend()
    plt.grid()
    plt.savefig(f'./img/{data_name}_p_r_{model_name}_{time_name}.png', bbox_inches='tight')
#         plt.close(fig)
    plt.show()

def roc_curve_plot(y_test, pos_proba, data_name, model_name=None,time_name=None): # pred_proba positive값
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
    
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
    # 임곗값에 따른 FPR, TPR 값을 반환 받음.
    fprs, tprs, thresholds = roc_curve(y_test, pos_proba)

    fig = plt.figure(figsize=(8, 6))
    # ROC Curve를 plot 곡선으로 그림.
    plt.plot(fprs, tprs, label='ROC')
    # 가운데 대각선 직선을 그림.
    plt.plot([0, 1], [0, 1], 'k--', label='Random')

    # FPR X 축의 Scale을 0.1 단위로 변경, X,Y 축명 설정등
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.title(f'Model ROC Curve {data_name} {model_name}')
    plt.xlabel('FPR( 1 - Sensitivity )')
    plt.ylabel('TPR( Recall )')
    plt.legend()
    plt.savefig(f'./img/{data_name}_roc_{model_name}_{time_name}.png', bbox_inches='tight')
#         plt.close(fig)
    plt.show()

 # 결정트리 모형도 sklearn 0.23.1
def Plot_tree_(data_name, model,model_name,time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)

    plt.figure(figsize=(12,8))
    tree.plot_tree(model, filled=True)
    plt.savefig(f"./imgTree/{data_name}_tree_{model_name}_{time_name}.png", bbox_inches='tight')
#     plt.close(fig)
    plt.show()
    
 # 결정트리에서 feature_importances_ # gridcv에는 없음
def Plot_feature_importance(data_name, model, indexs, model_name, time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)

    df = pd.Series(model.feature_importances_, index=indexs)
    df_20 = df.sort_values(ascending=False)#[:20]
    plt.figure(figsize=(8,6))
    sns.barplot(x=df_20, y=df_20.index)
    plt.title(f'{model_name} feature_importance')
    plt.savefig(f"./imgSel/{data_name}_{model_name}_{time_name}_FI.png", bbox_inches='tight') # dpi=100
#     plt.close(fig)
    plt.show()    
    
# LogisticRegression에서 사용 coef_
def Plot_coef_logit(data_name, model, indexs, model_name, time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)
#     importance = model.coef_[0]
#     for i,v in enumerate(importance):
#         print('Feature: %0d, Score: %.5f' % (i,v))
    
    df = pd.Series(model.coef_[0], index=indexs)
    df_20 = df.sort_values(ascending=False)#[:20]
    plt.figure(figsize=(8,6))
    sns.barplot(x=df_20, y=df_20.index)
#     plt.bar([ x for x in range(len(importance))], importance)
    plt.title(f'{model_name} 회귀계수(coef_)')
    plt.savefig(f"./imgSel/{data_name}_{model_name}_{time_name}_Coef.png", bbox_inches='tight') # dpi=100
#     plt.close(fig)
    plt.show() 
    
def Plot_permutation_importance(data_name, importance, indexs, model_name, time_name):
    # 한글깨짐 윈도우(window)
    font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name() # 폴더구분자 python / 임
    mpl.rc('font', family=font_name) # ahronbd.ttf
#     mpl.rcParams['font.family'] = 'NanumGothic'
#     mpl.rc('axes', unicode_minus=False)

    df = pd.Series(importance, index=indexs)
    df_20 = df.sort_values(ascending=False)#[:20]
    plt.figure(figsize=(8,6))
    sns.barplot(x=df_20, y=df_20.index)
    plt.title(f'{model_name} permutation_importance')
    plt.savefig(f"./imgSel/{data_name}_{model_name}_{time_name}_PI.png", bbox_inches='tight') # dpi=100
#     plt.close(fig)
    plt.show() 

In [5]:
# 학습곡선
def Plot_learning_curve(data_name, model, model_name, time_name, X, y, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
            model, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    plt.figure(figsize=(8,6))
    plt.title(f'{data_name} {model_name} learning curve')
    plt.grid()
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",label="Training")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",label="Cross-validation")
    plt.legend(loc="best")
    plt.xlabel("Training ")
    plt.ylabel("Score")
    plt.savefig(f"./imgLearnCurve/{data_name}_l_c_{model_name}_{time_name}.png", bbox_inches='tight')
#     plt.close(fig)
    plt.show()

In [6]:
# cv_results_데이터로 GridSearchCV 곡선
def plot_grid_curve(model, grid_params, data_name, model_name,time_name):
    df = pd.DataFrame(model.cv_results_)
    results = ['mean_test_score','mean_train_score'] 
    fig, axes = plt.subplots(1, len(grid_params), figsize = (3*len(grid_params), 4) )  # sharey='row'
    axes[0].set_ylabel("Score", fontsize=10)
    for idx, (param_name, param_range) in enumerate(grid_params.items()):
        grouped_df = df.groupby(f'param_{param_name}')[results].agg({'mean_train_score': 'mean', 'mean_test_score': 'mean'})
        previous_group = df.groupby(f'param_{param_name}')[results]
        axes[idx].set_xlabel(param_name, fontsize=10)
#         axes[idx].set_ylim(0.0, 1.1)
        lw = 2 # 선의 굵기
        axes[idx].plot(param_range, grouped_df['mean_train_score'], label="Training score", color="darkorange", lw=lw)
        axes[idx].plot(param_range, grouped_df['mean_test_score'], label="Cross-validation score",  color="navy", lw=lw)
#     handles, labels = axes[0].get_legend_handles_labels()
    fig.suptitle(f'{data_name} {model_name} Validation curves', fontsize=20)
    plt.legend(loc = "best")
    fig.savefig(f"./imgGridCurve/{data_name}_g_v_{model_name}_{time_name}.png", bbox_inches='tight')
    fig.subplots_adjust(bottom=0.25, top=0.85)  
#     plt.close(fig)
    plt.tight_layout()
    plt.show()   

### 2.  metric 함수
#### 2-1 다중분류

In [7]:
# 다중분류에서는 confusion_matrix, accuracy, roc_auc 만 나온다
def get_clf_eval_poly(y_test, pred, pos_proba=None):
    metrics_dict={}
    confusion = confusion_matrix( y_test, pred)
    metrics_dict['confusion_matrix'] = confusion.tolist() # confusion nd.array  --> list
    accuracy = accuracy_score(y_test , pred)
    metrics_dict['accuracy'] = accuracy
    roc_auc = roc_auc_score(y_test, pos_proba) 
    metrics_dict['auc_score'] = auc_score
    print(f'오차 행렬 :\n {confusion}')
    print(f" 정확도: {accuracy:.4f}, auc_score:{roc_auc:.4f}")
    return metrics_dict

#### 2-2 이진분류

In [8]:
# 다중분류에서는 confusion_matrix, accuracy만 나온다
# 이진분류에서 성능평가지표 종합 함수
def get_clf_eval(y_test=None, pred=None, pos_proba=None): # 권철민 저 책자에 누락 pred_proba
    metrics_dict={}
    confusion = confusion_matrix(y_test, pred)
    metrics_dict['confusion_matrix'] = confusion.tolist()
    accuracy = accuracy_score(y_test , pred)
    metrics_dict['accuracy'] = accuracy
    precision = precision_score(y_test , pred)
    metrics_dict['precision'] = precision
    recall = recall_score(y_test , pred )     
    metrics_dict['recall'] = recall
    f1 = f1_score(y_test, pred )
    metrics_dict['f1'] = f1
    roc_auc = roc_auc_score(y_test, pos_proba) # 권철민 저 책자에 오기 pred --> pred_proba  # average='macro'
    metrics_dict['roc_auc'] = roc_auc
    # print(f'오차 행렬 :\n {confusion}')
    # print(f" 정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, auc_score:{roc_auc:.4f}")
    print(f" 정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, f1_score: {f1:.4f}, auc_score:{roc_auc:.4f}")
    return metrics_dict

### 3. model train, metric호출

In [9]:
# get_params()관련 LGBM기준 (XGB유사) # sklearn version별 api 다름
# max_depth log2(데이터수)  # pima 768 => 10  # customer 76020 => 16 # credit 284807 => 18 # project 797만=>22
# num_leaves  : 2^(max_depth) 작아야 함  최대 리프노드 갯수 = max_leaf_nodes(dt, rf)
# min_child_samples : Leaf node가 되기 위한 최소한 데이터 개체수  = min_samples_leaf  # min_samples_split (dt,rf 분리 최소 데이터)
# min_child_weight = min_child_leaf (dt, rf) *gbm은 없음    
# subsample (gbm,xgb,lgbm) 데이터 샘플링 비율 *lgbm은 subsample_for_bin 20000, subsample_freq 0
# colsample_bytree = max_features (dt, rf, gbm) 개별 트리를 학습할 때마다 무작위로 선택하는 feature 비율을 제어 (feature 많을 경우) * 
    # XGB : colsample_bylevel, colsample_bynode, colsample_bytree  # pima 8 # customer 371 # credit 31  # project 25
# reg_alpha : L1 규제(제거) # reg_lambda : L2 규제(최소화) 
# 영혼까지 learning_rate 내리고,  n_estimators(rf,gbm,xgb,lgbm) 올리고
def get_params(model_name, data_name):  # param_grid 2개항목 이상일때만 plot_grid_curve 가능
    if (grid == 2) : # RandomSearch    속도느링 LogisticRegression  GradientBoostingClassifier
        if model_name == 'LogisticRegression': # 속도느링 #"solver":['lbfgs','auto']
            param_grid = { 'C': stats.loguniform(1e-1, 1e0),'l1_ratio': stats.uniform(0, 1), "max_iter":stats.randint(100,500) }
        elif model_name == 'LinearSVC': # 'loss' ['hinge',''] # C 커지면 곡선 -> 직선 # tol(허용오차)
            param_grid = { 'C': stats.loguniform(1e-1, 1e0),"max_iter":stats.randint(100,500) }
        elif model_name == 'KNeighborsClassifier':#'algorithm':'auto' # p:1 manhattan_distance, 2 euclidean_distance 
            param_grid ={'n_neighbors':[5,10],'p':[1,2]}
        elif model_name == 'SGDClassifier':#early_stopping:False,'learning_rate': 'optimal','penalty': 'l2','loss': 'hinge','eta0':[0.0,1.0],
            param_grid = { 'alpha':stats.loguniform(1e-4,1e0),'l1_ratio':stats.uniform(0,1),'average':[True,False],
                            'epsilon':stats.loguniform(1e-1, 1e1),'max_iter':stats.randint[100,500]}
        elif model_name == 'DecisionTreeClassifier': 
            if(data_name=='project'):param_grid = {"max_depth": stats.randint(5, 50), "max_features": stats.randint(2, 24),
                "min_samples_split": stats.randint(10, 100), "min_samples_leaf": stats.randint(10, 100),"criterion": ["gini", "entropy"]}
        elif model_name == 'RandomForestClassifier': # n_estimators=100
            if(data_name=='project'):param_grid = {"max_depth":  stats.randint(5, 50),"max_features":stats.randint(2, 24),
            "min_samples_split":stats.randint(10,100),"min_samples_leaf":stats.randint(10,100),"bootstrap":[True,False],"criterion":["gini","entropy"]}
        elif model_name == 'GradientBoostingClassifier':#속도느림 #n_estimators=100 learning_rate:0.1 subsample:1.0 ccp_alpha:0.0 n_features
            if(data_name=='project'):param_grid={ 'max_depth' : stats.randint(5, 50),'min_samples_leaf': stats.randint(10,500),
                                                   'max_features':  stats.randint(5,24) }
        elif model_name == 'XGBClassifier': #min_child_samples, num_leaves 없음 #n_estimators=100 learning_rate:0.1 
            if(data_name=='project'):param_grid={'n_estimators': stats.randint(200, 1000),'learning_rate': stats.uniform(0.01, 0.6),
            'subsample':stats.uniform(0.3, 0.9),"max_depth":stats.randint(5,50),'colsample_bytree':stats.uniform(0.5,0.9),'min_child_weight':stats.loguniform(1e-3, 1e3) }
        elif model_name == 'LGBMClassifier':#n_estimators=100 learning_rate:0.1
            if(data_name=='project'):param_grid={'n_estimators': stats.randint(200, 1000),'learning_rate': stats.uniform(0.01, 0.6),
            'min_child_samples':stats.randint(10,500),'colsample_bytree':stats.uniform(loc=0.4,scale=0.6),'num_leaves': stats.randint(32,1000),
                                    'subsample': stats.uniform(loc=0.2, scale=0.8), 'min_child_weight': stats.loguniform(1e-3, 1e3),
                                      'reg_alpha': stats.loguniform(1e-3, 1e2),'reg_lambda': stats.loguniform(1e-3, 1e3)}
        else:
            param_grid = None
        return param_grid

    elif (grid == 3) : #GridSearch
        if model_name == 'LogisticRegression': # "solver":['lbfgs','auto']
            param_grid  = {'C': [0.1,0.2,0.3],"l1_ratio": [0.4,0.6,0.8],"max_iter":[400,500,600]}
        elif model_name == 'LinearSVC': # 'loss' ['hinge',''] # C 커지면 곡선 -> 직선 # tol(허용오차)
            param_grid ={'C': [0.2,0.3,0.4],"max_iter":[200,300,400] }
        elif model_name == 'KNeighborsClassifier':#'algorithm':'auto' # p:1 manhattan_distance, 2 euclidean_distance 
            param_grid ={'n_neighbors':[5,10],'p':[1,2]}
        elif model_name == 'SGDClassifier':#early_stopping:False,'learning_rate': 'optimal','penalty': 'l2','loss': 'hinge','eta0':[0.0,1.0],
            param_grid ={'alpha': [0.0001,0.01,0.1],"l1_ratio":[0.1,0.2,0.3],'epsilon':[0.1,1,10],'max_iter':[100,300,500]} # 빠름
        elif model_name == 'DecisionTreeClassifier': 
            if(data_name=='pima'):param_grid={'max_depth':[5,10,15],'min_samples_leaf':[10,100,500],'min_samples_split':[10,15,20],'max_features':[3,5,8]}
            if(data_name=='customer'):param_grid={'max_depth':[12,16,20],'min_samples_leaf':[10,16,20],'max_features':[100,200,300]}
            if(data_name=='credit'):param_grid={'max_depth':[12,18,24],'min_samples_leaf':[10,50,100],'max_features':[100,500,1000]}
            if(data_name=='project'):param_grid={'max_depth':[15,20,25],'min_samples_leaf':[50,100,150],'max_features':[4,8,12]}
        elif model_name == 'RandomForestClassifier': # n_estimators=100
            if(data_name=='pima'):param_grid={'max_depth':[5,10,15],'min_samples_leaf':[10,100,500],'min_samples_split':[10,15,20],'max_features':[3,5,8]}
            if(data_name=='customer'):param_grid={'max_depth':[12,16,20],'min_samples_leaf':[10,100,500],'max_features':[100,200,300]}
            if(data_name=='credit'):param_grid={'max_depth':[12,18,24],'min_samples_leaf':[10,50,100],'max_features':[100,500,1000]}
            if(data_name=='project'):param_grid={'max_depth':[15,20,25],'min_samples_leaf':[20,40,60],'min_samples_split':[20,40,60]}
        elif model_name == 'GradientBoostingClassifier':#속도느림 #n_estimators=100 learning_rate:0.1 subsample:1.0 ccp_alpha:0.0 n_features
            if(data_name=='pima'):param_grid={'max_depth':[10],'min_samples_leaf':[10,15],'min_samples_split':[10,20],'max_features':[3,5,8]}
            if(data_name=='customer'):param_grid={'max_depth':[16],'min_samples_leaf':[10,100],'max_features':[100,200,300]}
            if(data_name=='credit'):param_grid={'max_depth':[18],'min_samples_leaf':[10,100],'max_features':[10,20]}
            if(data_name=='project'):param_grid={'max_depth':[5,10,20],'min_samples_leaf':[100,200],'max_features':[5,10,15]}
        elif model_name == 'XGBClassifier': #min_child_samples, num_leaves 없음 #n_estimators=100 learning_rate:0.1 
            if(data_name=='pima'):param_grid={'max_depth':[5,10,15],'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75,1],
                                  'subsample':[0.8,1],'reg_alpha': [0,0.5],'reg_lambda':[0.5,1]}
            if(data_name=='customer'):param_grid={'max_depth':[12,16,20],'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75,1],
                                  'subsample':[0.8,1],'reg_alpha': [0,0.5],'reg_lambda':[0.5,1]}
            if(data_name=='credit'):param_grid={'max_depth':[18,36],'min_child_weight':[1,3],'colsample_bytree':[0.5,1]}
                                  #'subsample':[0.8,1]} #'reg_alpha': [0,0.5],'reg_lambda':[0.5,1]}
            if(data_name=='project'):param_grid={'n_estimators':[400,500,600],'learning_rate':[0.03,0.05,0.07],'max_depth':[6,15,30],
             'min_child_weight':[0.1,1,3],'colsample_bytree':[0.5,0.75,0.95],'subsample':[0.5,0.7,0.9],'reg_alpha':[0,0.5,1],'reg_lambda':[0,0.5,1]}
        elif model_name == 'LGBMClassifier':#n_estimators=100 learning_rate:0.1
            if(data_name=='pima'):param_grid={'max_depth':[128,160],'min_child_samples':[10,100,500],'colsample_bytree': [0.5,0.75,1],
                                  'num_leaves':[32,128,512],'subsample':[0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
            if(data_name=='customer'):param_grid={'max_depth':[128,512],'min_child_samples':[10,100,500],'colsample_bytree': [0.5,0.75,1],
                                  'num_leaves':[32,128,512],'subsample':[0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
            if(data_name=='credit'):param_grid={'max_depth':[128,512,1024],'min_child_samples':[10,100,1000],'colsample_bytree': [0.5,0.75,1],
                                  'num_leaves':[32,512,2048],'subsample':[0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
            if(data_name=='project'):param_grid={'n_estimators':[400,500,600],'learning_rate':[0.03,0.05,0.07], 'min_child_samples':[100,150,200],
              'colsample_bytree':[0.5,0.75,1],'num_leaves':[50,100,150],'subsample':[0.6,0.8,1],'reg_alpha': [0,0.5,1],'reg_lambda':[0,0.5,1]}
                
        else:
            param_grid = None
        return param_grid                                               

In [19]:
# 모델과 학습/테스트 데이터 셋을 입력하면 학습 -> GridSerchCV 수행 + 성능 평가(metric) 등 반환
def get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test,eval_X=None,eval_y=None,is_expm1=False):
# def get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test,is_expm1=False):
    print('###',model.__class__.__name__,'###')
    if (grid == 2) :  # RandomSearchCV
        param_grid  = get_params(model_name, data_name)
        print(f'RandomSearchCV 입력 파라미터:{param_grid}')
        # 회귀, 분류에 따라 scoring 변경 # 분류 : accuracy, f1, neg_log_loss, roc_auc
        model=RandomizedSearchCV(model,param_distributions=param_grid,scoring='roc_auc',return_train_score=True,n_iter=5)
        model.fit(X_train, y_train) # 랜덤포레스트는 early_stopping_rounds없음 * xgb, lgbm, sgb만 있음
#         model.fit(X_train, y_train,early_stopping_rounds=10,eval_metric="logloss",eval_set=[(eval_X,eval_y)],verbose=True)
        best_params = model.best_params_
        print(f'RandomSearchCV 최적 파라미터:{ best_params}\n')
        # grid_curve plot 
#         plot_grid_curve(model, param_grid, data_name, model_name, time_name)
        # best_estimator_
        model = model.best_estimator_
        pred = model.predict(X_test)
        if model_name == 'LinearSVC':
            pred_proba = model._predict_proba_lr(X_test) 
        else:
            pred_proba = model.predict_proba(X_test)   
        if is_expm1 : #is_expm1 <= log변환시 복원
            y_test = np.expm1(y_test)
            pred = np.expm1(pred)  
        metrics_dict = get_clf_eval(y_test, pred, pred_proba[:,1])
        spend_metric_time=(time.time() - start_model_time)
        print(f'{model_name} metric 수행시간 : {int(spend_metric_time//60)}분 {spend_metric_time%60:.2f}초\n')
        if (plot == 1) :
            Plot_learning_curve(data_name,model,model_name,time_name,X_train,y_train,n_jobs=-1,train_sizes=np.linspace(.1,1.0,5))
            precision_recall_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name )
            roc_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name)
            if model_name in ['LogisticRegression','SGDClassifier','LinearSVC']:
                Plot_coef_logit(data_name, model, X_train.columns, model_name, time_name) # 회귀계수 
            else : 
    # [KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,GradientBoostingClassifier,XGBClassifier,LGBMClassifier]:
                Plot_feature_importance(data_name, model, X_train.columns, model_name, time_name) # 회귀트리 # lr x
            # permutation_importance
            results = permutation_importance(model, X_test, y_test, scoring='f1')
            importance = results.importances_mean
            Plot_permutation_importance(data_name, importance, X_train.columns, model_name, time_name)
        return model, best_params, metrics_dict

    elif (grid == 3) : # GridSearchCV
        param_grid  = get_params(model_name, data_name)
        print(f'GridSearchCV 입력 파라미터:{param_grid}')
        # 회귀, 분류에 따라 scoring 변경 # 분류 : accuracy, f1, neg_log_loss, roc_auc
        model = GridSearchCV(model, param_grid=param_grid, scoring='roc_auc', return_train_score=True) # cv=2 최소, default 5
#         gridcv = GridSearchCV(model, param_grid=param_grid, scoring=None, return_train_score=True)
#         model.fit(X_train, y_train) # 랜덤포레스트는 early_stopping_rounds없음 * xgb, lgbm, sgb만 있음
        model.fit(X_train, y_train,early_stopping_rounds=10,eval_metric="logloss",eval_set=[(eval_X,eval_y)],verbose=True)
        best_params = model.best_params_
        print(f'GridSearchCV 최적 파라미터:{ best_params}')
                                                 
        plot_grid_curve(model, param_grid, data_name, model_name, time_name)
        # best_estimator_
        model = model.best_estimator_
        pred = model.predict(X_test)
        if model_name == 'LinearSVC':
            pred_proba = model._predict_proba_lr(X_test) 
        else:
            pred_proba = model.predict_proba(X_test)   # LinearSVC 없음
        if is_expm1 : #is_expm1 <= log변환시 복원
            y_test = np.expm1(y_test)
            pred = np.expm1(pred)  
        metrics_dict = get_clf_eval(y_test, pred, pred_proba[:,1])
        spend_metric_time=(time.time() - start_model_time)
        print(f'{model_name} metric 수행시간 : {int(spend_metric_time//60)}분 {spend_metric_time%60:.2f}초\n')  
        if (plot == 1) :
            Plot_learning_curve(data_name,model,model_name,time_name,X_train,y_train,n_jobs=-1,train_sizes=np.linspace(.1,1.0,5))
            precision_recall_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name )
            roc_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name)
            if model_name in ['LogisticRegression','SGDClassifier','LinearSVC']:
                Plot_coef_logit(data_name, model, X_train.columns, model_name, time_name) # 회귀계수 
            else : 
        # [KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,GradientBoostingClassifier,XGBClassifier,LGBMClassifier]
                Plot_feature_importance(data_name, model, X_train.columns, model_name, time_name) # 회귀트리 # lr x
            results = permutation_importance(model, X_test, y_test, scoring='f1')
            importance = results.importances_mean
            Plot_permutation_importance(data_name, importance, X_train.columns, model_name, time_name)
        return model, param_grid, best_params, metrics_dict

    else: # grid = 1, 4
        model.fit(X_train, y_train)
    # early_stopping_rounds 기능은 xgboot LGBM ==> eval_metric: auc(bin only) "logloss":Negative lig-likehood multiclass "mlogloss",error,merror
#     model.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="logloss", eval_set=[(X_train, y_train), (X_test, y_test)], verbose=True)
#     model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=[(eval_X, eval_y)], verbose=True)
        ##########################################    
        # model 예측(에러) 확률    # 평가지표(metrics)을 위한 negitive, positive 확률로 변환
        pred = model.predict(X_test)
        if model_name == 'LinearSVC':
            pred_proba = model._predict_proba_lr(X_test) 
        else:
            pred_proba = model.predict_proba(X_test)   # LinearSVC 없음
        if is_expm1 : #target log1p 변환시 is_expm1 <= log변환시 복원
            y_train = np.expm1(y_train)
            y_test = np.expm1(y_test)
            pred = np.expm1(pred)  
        metrics_dict = get_clf_eval(y_test, pred, pred_proba[:,1])
        spend_metric_time=(time.time() - start_model_time)
        print(f'{model_name} metric 수행시간 : {int(spend_metric_time//60)}분 {spend_metric_time%60:.2f}초\n')  
        if (plot == 1) :
            Plot_learning_curve(data_name,model,model_name,time_name,X_train,y_train,n_jobs=-1,train_sizes=np.linspace(.1,1.0,5))
            precision_recall_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name )
            roc_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name)
            if model_name in ['LogisticRegression','SGDClassifier','LinearSVC']:
                Plot_coef_logit(data_name, model, X_train.columns, model_name, time_name) # 회귀계수 
            else : 
    # [KNeighborsClassifier,DecisionTreeClassifier,RandomForestClassifier,GradientBoostingClassifier,XGBClassifier,LGBMClassifier]:
                Plot_feature_importance(data_name, model, X_train.columns, model_name, time_name) # 회귀트리 # lr x
            results = permutation_importance(model, X_test, y_test, scoring='f1')
            importance = results.importances_mean
            Plot_permutation_importance(data_name, importance, X_train.columns, model_name, time_name)
        return model, metrics_dict        

## 1. data preprocessing, split

In [17]:
# 사전 데이터 가공
def get_preprocessed_df(df=None):
    ## 
    df_copy = df.copy()
#     df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else ( 2 if 100 <= x < 126 else 3))
#     df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
#     df_copy.drop('식전혈당(공복혈당)', axis=1, inplace=True)    
#     df_copy['연령'] =df_copy['연령대코드(5세단위)'].apply( lambda x: x*5 + 17 )
#     df_copy.drop('연령대코드(5세단위)', axis=1, inplace=True)
#     target_count = df_copy[ df_copy['target'] == 1 ].shape[0]
#     not_target_count = df_copy[ df_copy['target'] != 1 ].shape[0]
#     frac=round( (target_count/not_target_count), 3)
#     df_target = df_copy[ df_copy['target'] == 1 ]
#     df_X =  df_copy[ df_copy['target'] != 1 ].sample(frac=frac, random_state=1)
#     df_copy = pd.concat ([df_target, df_X]).sort_values(by=['기준년도'])
#     df_copy.drop(['기준년도','시도코드'], axis=1, inplace=True)    
#     df_copy = sklearn.utils.shuffle(df)
#     print(df_copy[df_copy['target'] == 1].shape[0])
#     print(df_copy[df_copy['target'] != 1].shape[0]) 
    ## 범주형 drop # 범주형 feature 10개
#     category_features = ['성별코드','청력(좌)','청력(우)', '요단백', '흡연상태', '음주여부', '구강검진수검여부']
#     df_copy.drop(category_features, axis=1, inplace=True)
    df_copy = pd.get_dummies(df_copy)
    return df_copy

# 1.필요시 전처리 리턴 => 2. X,y(target)분리 => 3. 필요시 scaling => 4. train,test로 split
def get_train_test_dataset(df=None):
    # 전체 df 전처리 필요시
    df = get_preprocessed_df(df)
    
    ## target과 train 데이터 분리
    # pima
#     y_target = df['Outcome']
#     X_features = df.drop('Outcome', axis=1)
    # customer(bank)
#     y_target = df['TARGET']
#     X_features = df.drop('TARGET', axis=1)    

    # credit
#     y_target = df['Class']
#     X_features = df.drop('Class', axis=1)    
#     scaler = StandardScaler()
#     amount_n = scaler.fit_transform(X_features['Amount'].values.reshape(-1, 1))
#     amount_n = np.log1p(X_features['Amount'])
#     X_features.insert(0, 'Amount_Scaled', amount_n)
#     X_features.drop(['Amount'], axis=1, inplace=True)
    
#     X_features = scaler.fit_transform(X_features)
#     X_features = np.log1p(X_features)

    #project
    y_target = df['target']
    X_features = df.drop('target', axis=1)
    columns = X_features.columns
#     scaler = MinMaxScaler()
#     scaler = StandardScaler()
#     X_features = scaler.fit_transform(X_features)
    X_features = np.log1p(X_features)
    X_features = pd.DataFrame(X_features, columns=columns)
    
    ## train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 Stratified 기반 분할
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156,stratify=y_target)
    # xgb,lgbm,sgb early_stopping_rounds 사용시 ==> test 데이터 eval_set 으로 다시 분리  # train, test 분리 0.3 필요
#     X_test, eval_X, y_test, eval_y = train_test_split(X_test, y_test, test_size=0.3, random_state=256, stratify=y_test)
    
    return X_train, X_test, y_train, y_test
#     return X_train, X_test, y_train, y_test, eval_X, eval_y

## 4. data 및 model selection, run

In [ ]:
# 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

# data_name = "breast_cancer"
# data_name = "pima"
# data_name = "customer"
# data_name = "credit"
data_name = "project104(1_1)LogOhe"

grid = 1      # 초기모델링    # 초기 model_list, # target scaler했다면 is_expm1=False, True 확인
# grid = 2     # RandomSearchCV 수행    # params 조정
# grid = 3     # GridSearchCV 수행     # params 조정
# grid = 4    # best params 적용  # 해당 model_list

plot = 0    # plot  disable 수행시간때문에
# plot = 1  # plot enable 

def run():    
    ## data 데이터 선택 ########################################    
#     df = dataset()
#     df = pima_df
#     df = cust_df
#     df = credit_df
    # project
#     df = pd.read_csv('./data/NHIS_total_model.csv')
    df = pd.read_csv('./data/NHIS_model_1.csv')
#     df = df.iloc[:1000, :]
    target_count = df[df['target'] == 1].shape[0]
    target_ex_count = df[df['target'] != 1].shape[0]
    print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
    print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")

   ## 1. 데이터 전처리(preprocessing), 분리(split) ##########################################
   # 랜덤 포레스트
#     X_train, X_test, y_train, y_test = get_human_dataset( )
    
    X_train, X_test, y_train, y_test = get_train_test_dataset(df)
#     X_train, X_test, y_train, y_test, eval_X, eval_y = get_train_test_dataset(df)
 
    ## model 선택 ###########################  LinearSVC() 개별에서
      # KNeighborsClassifier(n_jobs=-1), learning_curve loop
      # [ SVC(random_state=1, kernel="linear", C=1, probability=True)] # 시간 많이 소요
    model_list= [LogisticRegression(random_state=1, n_jobs=-1),
                 LinearSVC(random_state=1, loss='hinge'),
                 SGDClassifier(random_state=1,n_jobs=-1, loss='modified_huber'), 
                 DecisionTreeClassifier(random_state=1),
                 RandomForestClassifier(random_state=1,n_jobs=-1),
        GradientBoostingClassifier(random_state=1,validation_fraction=0.1,n_iter_no_change=20,subsample=0.25), # xgb,lgbm:fit조기중단가능
                 XGBClassifier(random_state=1,n_jobs=-1,silent=True,device='gpu'),
                 LGBMClassifier(random_state=1,n_jobs=-1) ]

#     model_list= [
#         LogisticRegression(random_state=1,n_jobs=-1,C=0.1,l1_ratio=0.1,max_iter=100,solver='lbfgs'),
#         KNeighborsClassifier(n_jobs=-1,n_neighbors=10,p=1),
#         SGDClassifier(random_state=1,n_jobs=-1,loss='modified_huber',alpha=0.0001,epsilon=0.1,eta0=0.0,l1_ratio=0.01,max_iter=100), 
#         DecisionTreeClassifier(random_state=1,max_depth=5,max_features=8,min_samples_leaf=20,min_samples_split=10),
#         RandomForestClassifier(random_state=1,n_jobs=-1,max_depth=5,max_features=5,min_samples_leaf=15,min_samples_split=10),
#         GradientBoostingClassifier(random_state=1,max_depth=10,max_features=5,min_samples_leaf=10,min_samples_split=10),
#         XGBClassifier(random_state=1,n_jobs=-1,silent=True,colsample_bytree=0.5,max_depth=15,min_child_weight=1,reg_alpha=0,reg_lambda=0.5,subsample=1),
#         LGBMClassifier(random_state=1,n_jobs=-1,colsample_bytree=1,max_depth=128,min_child_samples=5,num_leaves=64,reg_alpha=0,reg_lambda=0.5,subsample=0.8)
#     ]    

In [ ]:
def trainMetricSave(model_list)
    time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
    for model in model_list:
        start_model_time = time.time()
        model_name = model.__class__.__name__
        if(grid == 1 or grid == 4): # GridSerchCV 없이 초기 2. ~ 3. model 훈련, metric 리턴받아 처리 
            model, metrics_dict = \
                get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test)
        elif(grid == 2): # RandomSearchCV 수행 ==> 2. data split ~ 3. model 훈련, metric 리턴 ####
            model, best_params, metrics_dict =\
                        get_model_train_eval(model, model_name,data_name,time_name, X_train, X_test, y_train, y_test)
        elif(grid == 3): # GridSerchCV, RandomSearchCV 수행 ==> 2. data split ~ 3. model 훈련, metric 리턴 ####
            model, param_grid, best_params, metrics_dict =\
                        get_model_train_eval(model, model_name,data_name,time_name, X_train, X_test, y_train, y_test)
        ## metrics json 저장 
        metrics_total_dict = {}
        spend_model_time=(time.time() - start_model_time)
        metrics_total_dict['Experiment_date_time'] = time_name
        metrics_total_dict['data_name'] = data_name
        metrics_total_dict['model_name'] = model_name
        if(param_grid):        
            metrics_total_dict['param_grid'] = param_grid 
        if(best_params):        
            metrics_total_dict['best_params'] = best_params
        metrics_total_dict['model_params'] = model.get_params()
        metrics_total_dict['metrics'] =metrics_dict
        metrics_total_dict['Execution_time'] = spend_model_time
        with open(f'./json/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
            json.dump(metrics_total_dict, f, indent=4)            
        print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  
        if(best_params):
            params = 'random_state=1'
            for key, value in best_params.items():
                params = params + ',' + str(key) + '=' + str(value)
    #             print(f'{model_name} params : {params}\n')
            params_dict[f'{model_name}']=params
    spend_time=(time.time() - start_time)
    print(f'전체수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')  
    return 

In [18]:
    ##  2. data split ~ 3. 훈련, metric  ###################################################
    if (grid == 1): # GridSerchCV 없이 초기 2. ~ 3. model 훈련, metric 리턴받아 처리 ####
        time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
        for model in model_list:
            start_model_time = time.time()
            model_name = model.__class__.__name__
            model, metrics_dict = \
                get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test)
            ## metrics json 저장 
            metrics_total_dict = {}
            model_params = model.get_params() # dict
            spend_model_time=(time.time() - start_model_time)
            metrics_total_dict['Experiment_date_time'] = time_name
            metrics_total_dict['data_name'] = data_name
            metrics_total_dict['model_name'] = model_name
            metrics_total_dict['model_params'] = model_params
    #         metrics_total_dict['scaler'] = scaler
    #         metrics_total_dict['p_degree'] = p_degree
            metrics_total_dict['metrics'] =metrics_dict
    #         metric_dict['best_params'] = best_params
            metrics_total_dict['Execution_time'] = spend_model_time
            with open(f'./json1/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
                json.dump(metrics_total_dict, f, indent=4)            
            print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  
        spend_time=(time.time() - start_time)
        print(f'전체수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')  
        
    elif (grid == 2): # RandomSearchCV 수행 ==> 2. data split ~ 3. model 훈련, metric 리턴 ####
        time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
        params_dict={}
        for model in model_list:
            start_model_time = time.time()
            model_name = model.__class__.__name__
            model, best_params, metrics_dict =\
                        get_model_train_eval(model, model_name,data_name,time_name, X_train, X_test, y_train, y_test)
            ## metrics json 저장 
            metrics_total_dict = {}
            spend_model_time=(time.time() - start_model_time)
            metrics_total_dict['Experiment_date_time'] = time_name
            metrics_total_dict['data_name'] = data_name
            metrics_total_dict['model_name'] = model_name
#             metrics_total_dict['param_grid'] = param_grid   # gridsearchcv get_params() ==> json 저장에러
#             metrics_total_dict['best_params'] = best_params # gridsearchcv get_params()  ==> json 저장에러
            metrics_total_dict['model_params'] = model.get_params()
    #         metrics_total_dict['scaler'] = scaler
    #         metrics_total_dict['p_degree'] = p_degree
            metrics_total_dict['metrics'] =metrics_dict
            metrics_total_dict['Execution_time'] = spend_model_time
            with open(f'./json2RandomSearch/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
                json.dump(metrics_total_dict, f, indent=4)            
            print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  
            # best_params에 대한 model별 params 
            params = 'random_state=1'
            for key, value in best_params.items():
                params = params + ',' + str(key) + '=' + str(value)
    #             print(f'{model_name} params : {params}\n')
            params_dict[f'{model_name}']=params
        spend_time=(time.time() - start_time)
        print(f'전체 수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')
        print(params_dict)
       
    elif (grid == 3 ): # GridSerchCV, RandomSearchCV 수행 ==> 2. data split ~ 3. model 훈련, metric 리턴 ####
        time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
        params_dict={}
        for model in model_list:
            start_model_time = time.time()
            model_name = model.__class__.__name__
            model, param_grid, best_params, metrics_dict =\
                        get_model_train_eval(model, model_name,data_name,time_name, X_train, X_test, y_train, y_test)
            ## metrics json 저장 
            metrics_total_dict = {}
            spend_model_time=(time.time() - start_model_time)
            metrics_total_dict['Experiment_date_time'] = time_name
            metrics_total_dict['data_name'] = data_name
            metrics_total_dict['model_name'] = model_name
            metrics_total_dict['param_grid'] = param_grid   # gridsearchcv get_params() ==> json 저장에러
            metrics_total_dict['best_params'] = best_params # gridsearchcv get_params()  ==> json 저장에러
            metrics_total_dict['model_params'] = model.get_params()
    #         metrics_total_dict['scaler'] = scaler
    #         metrics_total_dict['p_degree'] = p_degree
            metrics_total_dict['metrics'] =metrics_dict
            metrics_total_dict['Execution_time'] = spend_model_time
            with open(f'./json3GridSearch/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
                json.dump(metrics_total_dict, f, indent=4)            
            print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  
            # best_params에 대한 model별 params 
            params = 'random_state=1'
            for key, value in best_params.items():
                params = params + ',' + str(key) + '=' + str(value)
    #             print(f'{model_name} params : {params}\n')
            params_dict[f'{model_name}']=params
        spend_time=(time.time() - start_time)
        print(f'전체 수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')
        print(params_dict)

    else: # grid=4 # GridSearchCV best_params에 대한 model별 수기적용하여, 2. ~ 3. model 훈련, metric저장, 과적합여부 plot
        time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
        for model in model_list:
            start_model_time = time.time()
            model_name = model.__class__.__name__
            model, metrics_dict = \
                    get_model_train_eval(model, model_name, data_name, time_name, X_train, X_test, y_train, y_test)
        ## metrics json 저장 
            metrics_total_dict = {}
            spend_model_time=(time.time() - start_model_time)
            metrics_total_dict['Experiment_date_time'] = time_name
            metrics_total_dict['data_name'] = data_name
            metrics_total_dict['model_name'] = model_name
            model_params = model.get_params() # dict
            metrics_total_dict['model_params'] = model_params
    #         metrics_total_dict['scaler'] = scaler
    #         metrics_total_dict['p_degree'] = p_degree
            metrics_total_dict['metrics'] =metrics_dict
    #         metric_dict['best_params'] = best_params
            metrics_total_dict['Execution_time'] = spend_model_time
            with open(f'./json4/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
                json.dump(metrics_total_dict, f, indent=4)            
            print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  
        spend_time=(time.time() - start_time)
        print(f'전체 수행시간 : {int(spend_time//60)}분 {spend_time%60:.2f}초\n')        
#     return metrics_total_dict  
run() 

target   count : 521858 (50.0)%
target외 count : 521754 (50.0)%
### LogisticRegression ###
 정확도: 0.7138, 정밀도: 0.7038, 재현율: 0.7385, f1_score: 0.7208, auc_score:0.7853
LogisticRegression metric 수행시간 : 1분 10.39초

LogisticRegression 수행시간 : 1분 6.57초

### LinearSVC ###
 정확도: 0.7144, 정밀도: 0.7060, 재현율: 0.7351, f1_score: 0.7203, auc_score:0.7857
LinearSVC metric 수행시간 : 5분 0.17초

LinearSVC 수행시간 : 3분 49.77초

### SGDClassifier ###
 정확도: 0.7106, 정밀도: 0.7164, 재현율: 0.6973, f1_score: 0.7067, auc_score:0.7844
SGDClassifier metric 수행시간 : 5분 16.19초

SGDClassifier 수행시간 : 0분 16.02초

### DecisionTreeClassifier ###
 정확도: 0.6334, 정밀도: 0.6344, 재현율: 0.6297, f1_score: 0.6320, auc_score:0.6334
DecisionTreeClassifier metric 수행시간 : 5분 35.90초

DecisionTreeClassifier 수행시간 : 0분 19.71초

### RandomForestClassifier ###
 정확도: 0.7289, 정밀도: 0.7101, 재현율: 0.7738, f1_score: 0.7405, auc_score:0.8001
RandomForestClassifier metric 수행시간 : 6분 8.86초

RandomForestClassifier 수행시간 : 0분 32.96초

### GradientBoostingClassifier ###
 정확도: 0.

In [ ]:
model_list= [
    LogisticRegression(random_state=1,n_jobs=-1,C=0.1,l1_ratio=0.1,max_iter=100,solver='lbfgs'),
    KNeighborsClassifier(n_jobs=-1,n_neighbors=10,p=1),
    SGDClassifier(random_state=1,n_jobs=-1,alpha=0.0001,epsilon=0.1,eta0=0.0,l1_ratio=0.01,max_iter=100), 
    DecisionTreeClassifier(random_state=1,max_depth=5,max_features=8,min_samples_leaf=20,min_samples_split=10),
    RandomForestClassifier(random_state=1,n_jobs=-1,max_depth=5,max_features=5,min_samples_leaf=15,min_samples_split=10),
    GradientBoostingClassifier(random_state=1,max_depth=10,max_features=5,min_samples_leaf=10,min_samples_split=10),
    XGBClassifier(random_state=1,n_jobs=-1,silent=True,colsample_bytree=0.5,max_depth=15,min_child_weight=1,reg_alpha=0,reg_lambda=0.5,subsample=1),
    LGBMClassifier(random_state=1,n_jobs=-1,colsample_bytree=1,max_depth=128,min_child_samples=5,num_leaves=64,reg_alpha=0,reg_lambda=0.5,subsample=0.8)
]

## 개별 model별 (1. (data, run),  4.(data split) 항목만 사용치 않고, 2.metric ~ 3. 훈련 사용)

In [ ]:
start_time = time.time()

# data_name = "breast_cancer"
# data_name = "pima"
# data_name = "cust"
# data_name = "credit"
data_name = "project"

# grid = 1
# grid = 2
grid = 3
# grid = 4

# df = dataset()
# df = pima_df
# df = cust_df
# df = credit_df


# df = pd.read_csv('data/NHIS_total_model.csv')
# df['target'] = df['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
# df['target'] = df['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else ( 2 if 100 <= x < 126 else 3))
# df.drop('식전혈당(공복혈당)', axis=1, inplace=True)    
# df['연령'] =df['연령대코드(5세단위)'].apply( lambda x: x*5 + 17 )
# df.drop('연령대코드(5세단위)', axis=1, inplace=True)

# target_count = df_copy[ df_copy['target'] == 1 ].shape[0]
# not_target_count = df_copy[ df_copy['target'] != 1 ].shape[0]
# frac=round( (target_count/not_target_count), 3)

# df_target = df_copy[ df_copy['target'] == 1 ]
# df_X =  df_copy[ df_copy['target'] != 1 ].sample(frac=frac, random_state=1)
# df = pd.concat ([df_target, df_X]).sort_values(by=['기준년도'])
# df.drop(['기준년도','시도코드'], axis=1, inplace=True)    
# df = sklearn.utils.shuffle(df)
# print(df[df['target'] == 1].shape[0])
# print(df[df['target'] != 1].shape[0]) 

df = pd.read_csv('data/NHIS_model_1.csv')
# df = df.iloc[:200000, :]
target_count = df[df['target'] == 1].shape[0]
target_ex_count = df[df['target'] != 1].shape[0]
print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")

# 범주형 feature 10개
category_features = ['성별코드','청력(좌)','청력(우)', '요단백', '흡연상태', '음주여부', '구강검진수검여부']
df.drop(category_features, axis=1, inplace=True)
y_target = df['target']
X_features = df.drop('target', axis=1)

columns = X_features.columns
# scaler = MinMaxScaler()
# scaler = StandardScaler()
X_features = np.log1p(X_features)
# X_features = scaler.fit_transform(X_features)
X_features = pd.DataFrame(X_features, columns=columns)

## train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 Stratified 기반 분할
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=156,stratify=y_target)
# xgb,lgbm,sgb early_stopping_rounds 사용시 ==> test 데이터 eval_set 으로 다시 분리  # train, test 분리 0.3 필요
X_test, eval_X, y_test, eval_y = train_test_split(X_test, y_test, test_size=0.3, random_state=256, stratify=y_test)
   
# LinearSVC 규제에 편향이 포함 scaler = StandardScaler 사용, loss=hinge(loss는 2개, squared-hinge)
## 아래 params에서 n_jobs=-1이 있는 곳은 그대로 추가 # xgb는 silent=True까지 추가
# model = LogisticRegression(random_state=1, n_jobs=-1)   # tree 사용안됨
# model = SVC(random_state=1, kernel="linear", C=1, probability=True)
# model = LinearSVC(random_state=1, loss='hinge')
# model = KNeighborsClassifier(n_jobs=-1)
# model = SGDClassifier(random_state=1,n_jobs=-1)
# model= GaussianNB()
# model = tree.DecisionTreeClassifier(random_state=1)
# model = DecisionTreeClassifier(random_state=1)
# model = RandomForestClassifier(random_state=1,n_jobs=-1)
# model = GradientBoostingClassifier(random_state=1),
model = XGBClassifier(random_state=1,silent=True,n_jobs=-1, device='gpu')
# model = LGBMClassifier(random_state=1,n_jobs=-1, device='gpu')

## SGDClassifier
#  clf : 'hinge'(default but error),'log','modified_huber','squared_hinge','perceptron',
    # hinge, squared_hinge는 probability return하지 않고 log, modified_huber 리턴함
# reg:'squared_loss','huber', 'epsilon_insensitive' or'squared_epsilon_insensitive'
#     model =  SGDClassifier(random_state=1,n_jobs=-1, loss='log')  # log => logit + SGD
    
time_name = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = model.__class__.__name__

# grid = 1, 4    
# model,metrics_dict = get_model_train_eval_(model,model_name,data_name,time_name, X_train, X_test, y_train, y_test)
# grid = 2 (RandomSearchCV)
# model,best_params,metrics_dict=get_model_train_eval(model, model_name,data_name,time_name, X_train, X_test, y_train, y_test)
model,best_params,metrics_dict=get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test,eval_X,eval_y)
# grid =3 (GridSearchCV)
# model,param_grid,best_params,metrics_dict=get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test)
# model,best_params,metrics_dict=get_model_train_eval(model,model_name,data_name,time_name,X_train,X_test,y_train,y_test,eval_X,eval_y)

## model 저장
# pickle.dump(model, open(f'./modeling/model_{model_name}_{time_name}.sav', 'wb'))

# plot
# Plot_learning_curve(data_name, model, model_name, time_name,  X_train, y_train, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5))
# precision_recall_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name )
# roc_curve_plot(y_test, pred_proba[:,1], data_name, model_name, time_name)
# Plot_tree_(data_name, model, model_name, time_name) # LogisticRegression, RandomForestClassifier 사용못함 # 수행시간 많이 걸림
# Plot_feature_importance(data_name, model, X_train.columns, model_name,time_name ) # LogisticRegression 사용못함
# Plot_coef_logit(data_name, model, X_train.columns, model_name,time_name ) # LogisticRegression

## metrics json 저장 
metrics_total_dict = {}
spend_model_time=(time.time() - start_time)
metrics_total_dict['Experiment_date_time'] = time_name
metrics_total_dict['data_name'] = data_name
metrics_total_dict['model_name'] = model_name
metrics_total_dict['param_grid'] = param_grid   # gridsearchcv get_params() ==> json 저장에러
metrics_total_dict['best_params'] = best_params # gridsearchcv get_params()  ==> json 저장에러
metrics_total_dict['model_params'] = model.get_params()
# metrics_total_dict['scaler'] = scaler
# metrics_total_dict['p_degree'] = p_degree
metrics_total_dict['metrics'] =metrics_dict
metrics_total_dict['Execution_time'] = spend_model_time
# with open(f'./json1/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
# with open(f'./json2RandomSearch/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
with open(f'./json3GridSearch/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
# with open(f'./json4/{data_name}_{model_name}_{time_name}.json', 'w', encoding='UTF-8-sig') as f:
    json.dump(metrics_total_dict, f, indent=4)            
print(f'{model_name} 수행시간 : {int(spend_model_time//60)}분 {spend_model_time%60:.2f}초\n')  


In [ ]:
0. data_name, df 선택 ==> target 분리, 전처리영역필요여부
1. 초기모델은 grid = 1 선택, 전체 model_list for문, 과적합 learning_curve plot 확인
    model_list,  Plot learning_curve
    
2. feature engineering : scaler, p_degree(다항식), 
    scaler log1p 변환시 is_expm1=True
    1. 3.항 실행    
    
3. GridSearchCV 사용 grid = 2 선택, 전체 model_list for문, parameter별 영향력 확인 grid_curve
    model_list,  plot grid_curve
    ==> 1항에서 parameter tunning 반복가능
    
4. grid = 3 best param model 선택, 과적합 learning_curve plot 확인
    model_list, Plot learning_curve
    
5. model과 최적 param 찾았다면, model
    model, Plot (coeff_, feature_importance_ )
    
6. xgb, lgbm의 early기능 사용시 eval_X, eval_y  

In [ ]:
### 데이터 로드
* get_train_test_dataset()에 의해 분리되지 않으면  
* get_preprocessed_df()활용
* get_human_dataset() 새로만들어서 리턴

###  데이터 분리, 전처리가 필요한가요?  get_train_test_dataset()   

### 모델 선택과  get_model_train_eval 
* 모델, param알고 있나요?  GridsearchCV 사용하여 최적 param찾으실 것인가요?
*  ==>  그냥 fit 선택, GridsearchCV 사용여부 결정 
    + 모델이 이진분류라면 custom_threshold 선택
* return 5개 custom_threshold, model, pred_proba, best_params, metric_lst 
* GridsearchCV나오고 best_param_에 의해 새롭게 입력되었으면 run함수의 모델에 최적의 param입력하고 
    - GridsearchCV 주석처리,  return값 best_params 확인
* GridsearchCV 사용하나요?
    + param 선택하였나요?
    + cv하신것인가요?
* run에서도 맞는 return 일치

### 모델 metric
* 이진분류이면 get_clf_eval(y_test, pred, pred_proba[:,1]) <=========== 6개(c,a,p,r,f1,auc)모두 
* 다항분류이면 get_clf_eval_poly(y_test, pred,pred_proba[:,1]) <==== confusion_matrix, accuracy, auc만 지원

### 그리기
* 이진트리 graphviz, boundary GridsearchCV 전후 사진이 필요한가요? (LogisticRegression 지원하지 않음)
    + 전후 저장장소의 파일이름을 달리 하세요 
* Plot 데이터 전처리 전후, 이진분류시 precision-recall curve
* tree
* feature_importances_
    + Plot1 이진분류
    + Plot_tree : sklearn 0.23.1
    + Plot_feature_importance_xgb_lgb ===> xgboost, gbm 맞게 import
### save
* metric json
* feature_importance image

In [ ]:
def dictParse(data):
    # with open(f'./static/json/{model_name}_{time_name}.json', 'r', encoding='UTF-8-sig') as json_file:
    #     data = json.load(json_file)
    #     print(data)
    if 'Experiment_date_time' in data.keys():
        time_name = data['Experiment_date_time']
    if 'model' in data.keys():
        model_n = data['model']
    if 'metrics' in data.keys():
        if 'confusion_matrix' in data['metrics'].keys():
            confusion = data['metrics']['confusion_matrix']
        if 'accuracy' in data['metrics'].keys():
            accuracy = data['metrics']['accuracy']
        if 'precision' in data['metrics'].keys():
            precision = data['metrics']['precision']
        if 'recall' in data['metrics'].keys():
            recall = data['metrics']['recall']    
        if 'f1' in data['metrics'].keys():
            f1_score = data['metrics']['f1']  
        if 'roc_auc' in data['metrics'].keys():
            auc_score = data['metrics']['roc_auc']  
    if 'Execution_time' in data.keys():
        Execution_time = data['Execution_time'] 
    # print(f'confusion_matrix{confusion}\n accuracy{accuracy}\n precision{precision}\n recall{recall}\n f1_score{f1_score}\n auc_score{auc_score}\n Execution_time{Execution_time}')
    return model_n, confusion, accuracy, precision, recall, f1_score, auc_score, Execution_time

# custom_threshold, metrics_total_dict = run()
metrics_total_dict = run()
model_n, confusion, accuracy, precision, recall, f1_score, auc_score, Execution_time = dictParse(metrics_total_dict)
print(f'model name {model_n}\n confusion_matrix{confusion}\n accuracy{accuracy}\n precision{precision}\n recall{recall}\n f1_score{f1_score}\n auc_score{auc_score}\n Execution_time{Execution_time}')

### data 관련

In [ ]:
def dataset():
    from sklearn.datasets import load_iris
    from sklearn.datasets import load_breast_cancer # xgb 유방암, voting사용
    from sklearn.datasets import load_diabetes  # pima아니고 
    from sklearn.datasets import load_boston # 회귀
    # built_in data 선택
    dataset = load_iris()
#     dataset = load_breast_cancer()    
    X_features= dataset.data
    y_label = dataset.target
    df = pd.DataFrame(data=X_features, columns=dataset.feature_names)
    df['target']= y_label
    return df

In [ ]:
# customer 은행고객 만족도 # xgb, gbm
cust_df = pd.read_csv('data/train_santander.csv', encoding='latin-1')
cust_df['var3'].replace(-999999,2, inplace=True)
# cust_df.info()

In [ ]:
# credit 신용카드 280,000중 사기 500건 : 0.178%
# 단계 1.default  2.'Time' feature remove  3.scaler=standard 'Amount' 4. scaler=lg1p 'Amount' + outlier 'V14'  5. GridSerchCV
def get_outlier_index(df = None, column=None, weight_tup=None):
    # fraud에 해당하는 column 데이터만 추출, 1/4 분위와 3/4 분위 지점을 np.percentile로 구함. 
    series = df[df['Class']==1][column] # 사기데이터
#     print(f'\nseries.name : {column}   weight : {weight_tup}')
    weight, lower_adj=weight_tup
    quantile_25 = np.percentile(series.values, 25)
#     print(f'quantile_25 : {quantile_25}')
    quantile_75 = np.percentile(series.values, 75)
#     print(f'quantile_75 : {quantile_75}')
    # IQR을 구하고, IQR에 weight를 곱하여 최대값과 최소값 지점 구함. 
    iqr = quantile_75 - quantile_25
#     print(f'iqr : {iqr}')
    iqr_weight = iqr * weight
#     print(f'iqr_weight : {iqr_weight}')
    # lower 조정계수 반영
    lowest_val = quantile_25 - iqr_weight + lower_adj
#     print(f'lowest_val : {lowest_val}')
    highest_val = quantile_75 + iqr_weight
#     print(f'highest_val : {highest_val}')
    # 최대값 보다 크거나, 최소값 보다 작은 값을 아웃라이어로 설정하고 DataFrame index 반환. 
    outlier_index = series[(series < lowest_val) | (series > highest_val)].index
#     print(f'outlier_index length : {len(outlier_index)}\n')
    return outlier_index
def remove_outlier(df):
    outlier_drop_indexs=[]
    outlier_index = get_outlier_index(df = credit_df, column='V14', weight_tup=(1.5, 0))  
    if (outlier_index.values.tolist() != []):
        outlier_drop_indexs.extend(outlier_index.values.tolist())
    print(f'len(outlier_drop_indexs) : {len(outlier_drop_indexs)}\n')
    df.drop(outlier_drop_indexs, axis=0, inplace=True)
    return df
credit_df = pd.read_csv('data/creditcard.csv')
# print(credit_df.info())
credit_df.drop('Time', axis=1, inplace=True)
credit_df = remove_outlier(credit_df)
# df.head(3)

In [ ]:
# pima diabetes
pima_df = pd.read_csv('data/diabetes.csv')
df=pima_df
#0 제거 후 nan 으로 변환 # np.nan의 type : float
col=['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI' ]
for i in col:
    df[i].replace(0, np.nan, inplace= True)
    # median 중앙값 찾기
def median_target(var):   
    temp = df[df[var].notnull()]
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    return temp
# o값 대체 NaN 채워넣기 
df.loc[(df['Outcome'] == 0 ) & (df['Glucose'].isnull()), 'Glucose'] = median_target('Glucose')[ median_target('Glucose')['Outcome']==0]['Glucose'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['Glucose'].isnull()), 'Glucose'] = median_target('Glucose')[ median_target('Glucose')['Outcome']==1]['Glucose'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = median_target('BloodPressure')[ median_target('BloodPressure')['Outcome']==0]['BloodPressure'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = median_target('BloodPressure')[ median_target('BloodPressure')['Outcome']==1]['BloodPressure'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = median_target('SkinThickness')[ median_target('SkinThickness')['Outcome']==0]['SkinThickness'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = median_target('SkinThickness')[ median_target('SkinThickness')['Outcome']==1]['SkinThickness'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['Insulin'].isnull()), 'Insulin'] =  median_target('Insulin')[ median_target('Insulin')['Outcome']==0]['Insulin'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['Insulin'].isnull()), 'Insulin'] = median_target('Insulin')[ median_target('Insulin')['Outcome']==1]['Insulin'].values[0]
df.loc[(df['Outcome'] == 0 ) & (df['BMI'].isnull()), 'BMI'] = median_target('BMI')[ median_target('BMI')['Outcome']==0]['BMI'].values[0]
df.loc[(df['Outcome'] == 1 ) & (df['BMI'].isnull()), 'BMI'] = median_target('BMI')[ median_target('BMI')['Outcome']==1]['BMI'].values[0]
# df.isnull().sum()
pima_df=df
# pima_df.info()

In [ ]:
# 임계값에 따른 성능평가지표 산출
def get_eval_by_threshold(y_test , pred_proba, thresholds):
    metrics_lst=[]
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba) 
        custom_predict = binarizer.transform(pred_proba) # predict 수행한 것과 동일
#         print('임곗값:', custom_threshold)
        metrics_dict = get_clf_eval(y_test , custom_predict, pred_proba)
        # print(metrics_dict)
        metrics_lst.append(metrics_dict)
    return metrics_lst

     ###################### 사용자임계값 사용시 binary classification 최적의 임계값 선택 ####################################
#     thresholds =np.arange(0.1, 0.5, 0.02).tolist()
#     metrics_lst = get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds )
#     # # print(metrics_lst)

#     f1_lst=[]
#     for metric_dict in metrics_lst:
#     #     # print(f'metric_dict : {metric_dict}')
#         if 'f1' in metric_dict:
#             f1_lst.append(metric_dict['f1'])     
#     # # print(f'f1_lst: {f1_lst}')
#     custom_threshold = thresholds[f1_lst.index(max(f1_lst))]
#     print(f'best threshold : {custom_threshold}')

#     # # 최적 임계값으로 설정한 Binarizer 생성 # 원래 default 임계값 0.5
#     binarizer = Binarizer(threshold=custom_threshold)
#     pred_custom_th = binarizer.fit_transform(pred_proba[:, 1].reshape(-1,1)) 

    ##### return #############################################       
    # 사용자임계값 사용시 : 이진분류일때 pred_custom_th
#     return custom_threshold, model, pred_proba, get_clf_eval(y_test, pred_custom_th, pred_proba[:,1]) # 이진분류일때
#     return custom_threshold, model, pred_proba,  best_params, get_clf_eval(y_test, pred_custom_th, pred_proba[:,1])
    